In [33]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agent_name = "deep_research_scoping_agent"




In [ ]:
# Load .env file and create AWS Systems Manager secure parameters
from pathlib import Path
import dotenv

env_path = Path("/workspaces/agent_core_deep_research/.env")

if not env_path.exists():
    print(f"❌ .env file not found at {env_path}")
else:
    # Load environment variables
    env_vars = dotenv.dotenv_values(env_path)
    
    print(f"Found {len(env_vars)} environment variables in .env")
    
    # Create SSM client
    ssm_client = boto_session.client("ssm", region_name="us-west-2")
    
    # Create secure parameters for each key
    for key, value in env_vars.items():
        if not value or value.strip() == "":
            print(f"⚠️  Skipping empty value for {key}")
            continue
            
        parameter_name = f"/deep_research_scoping_agent/{key}"
        
        try:
            ssm_client.put_parameter(
                Name=parameter_name,
                Value=value,
                Type="SecureString",
                Overwrite=True,
                Description=f"Environment variable for deep_research_scoping_agent"
            )
            print(f"✅ Created/Updated secure parameter: {parameter_name}")
        except Exception as e:
            print(f"❌ Failed to create parameter {parameter_name}: {e}")
    
    print(f"\n✅ All environment variables stored as secure parameters in us-west-2")# Build Docker image and push to ECR


Found 6 environment variables in .env
✅ Created/Updated secure parameter: /deep_research_scoping_agent/TAVILY_API_KEY
✅ Created/Updated secure parameter: /deep_research_scoping_agent/OPENAI_API_KEY
✅ Created/Updated secure parameter: /deep_research_scoping_agent/ANTHROPIC_API_KEY
✅ Created/Updated secure parameter: /deep_research_scoping_agent/TAVILY_API_KEY
✅ Created/Updated secure parameter: /deep_research_scoping_agent/OPENAI_API_KEY
✅ Created/Updated secure parameter: /deep_research_scoping_agent/ANTHROPIC_API_KEY
✅ Created/Updated secure parameter: /deep_research_scoping_agent/LANGSMITH_API_KEY
✅ Created/Updated secure parameter: /deep_research_scoping_agent/LANGSMITH_TRACING
✅ Created/Updated secure parameter: /deep_research_scoping_agent/LANGSMITH_PROJECT

✅ All environment variables stored as secure parameters in us-west-2
✅ Created/Updated secure parameter: /deep_research_scoping_agent/LANGSMITH_API_KEY
✅ Created/Updated secure parameter: /deep_research_scoping_agent/LANGSMITH

#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 884B done
#1 DONE 0.0s

#2 [internal] load metadata for ghcr.io/astral-sh/uv:python3.12-bookworm-slim
#2 DONE 0.4s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/7] FROM ghcr.io/astral-sh/uv:python3.12-bookworm-slim@sha256:b64bb25505ad7d6630f185a0f5ab29dab437a5b99f522885c1665842fb0c304b
#4 resolve ghcr.io/astral-sh/uv:python3.12-bookworm-slim@sha256:b64bb25505ad7d6630f185a0f5ab29dab437a5b99f522885c1665842fb0c304b done
#4 DONE 0.0s

#5 [internal] load build context
#2 DONE 0.4s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/7] FROM ghcr.io/astral-sh/uv:python3.12-bookworm-slim@sha256:b64bb25505ad7d6630f185a0f5ab29dab437a5b99f522885c1665842fb0c304b
#4 resolve ghcr.io/astral-sh/uv:python3.12-bookworm-slim@sha256:b64bb25505ad7d6630f185a0f5ab29dab437a5b99f522885c1665842fb0c304b

Tagging image for ECR: 482387069690.dkr.ecr.us-west-2.amazonaws.com/deep_research_scoping_agent:latest...
Pushing image to ECR: 482387069690.dkr.ecr.us-west-2.amazonaws.com/deep_research_scoping_agent:latest...
The push refers to repository [482387069690.dkr.ecr.us-west-2.amazonaws.com/deep_research_scoping_agent]
f53431374520: Waiting
f130974e425e: Waiting
469aebfb7b90: Waiting
73c4402090ca: Waiting
2cc0a8e7ed73: Waiting
927305b24ea9: Waiting
4d940c864af5: Waiting
55d3e1734def: Waiting
be36e825b816: Waiting
e75f0adcf796: Waiting
05ccb1b61fbc: Waiting
162e72af9357: Waiting
4d940c864af5: Waiting
55d3e1734def: Waiting
be36e825b816: Layer already exists
e75f0adcf796: Layer already exists
05ccb1b61fbc: Layer already exists
162e72af9357: Layer already exists
f53431374520: Layer already exists
f130974e425e: Layer already exists
469aebfb7b90: Layer already exists
73c4402090ca: Layer already exists
2cc0a8e7ed73: Layer already exists
927305b24ea9: Layer already exists
f53431374520: Waiting
f130

In [34]:


import subprocess

# Get AWS account ID
account_id = boto_session.client("sts").get_caller_identity()["Account"]

# ECR repo URI
ecr_repo = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{agent_name}"

# Create ECR repository if it doesn't exist
print(f"Creating ECR repository: {agent_name}...")
ecr_client = boto_session.client("ecr")
try:
    ecr_client.create_repository(
        repositoryName=agent_name,
        imageScanningConfiguration={'scanOnPush': True}
    )
    print(f"✅ ECR repository '{agent_name}' created successfully")
except ecr_client.exceptions.RepositoryAlreadyExistsException:
    print(f"ℹ️  ECR repository '{agent_name}' already exists")

# Authenticate Docker to ECR
print("Authenticating to ECR...")
login_password = subprocess.run(
    ["aws", "ecr", "get-login-password", "--region", region],
    capture_output=True,
    text=True,
    check=True
).stdout

subprocess.run(
    ["docker", "login", "--username", "AWS", "--password-stdin", f"{account_id}.dkr.ecr.{region}.amazonaws.com"],
    input=login_password,
    text=True,
    check=True
)

# Build Docker image (run from parent directory)
print(f"Building Docker image: {agent_name}...")
subprocess.run(
    ["docker", "build", "-f", "src/Dockerfile", "-t", agent_name, "."],
    cwd="/workspaces/agent_core_deep_research",
    check=True
)

# Tag Docker image
print(f"Tagging image for ECR: {ecr_repo}:latest...")
subprocess.run(
    ["docker", "tag", f"{agent_name}:latest", f"{ecr_repo}:latest"],
    check=True
)

# Push Docker image to ECR
print(f"Pushing image to ECR: {ecr_repo}:latest...")
subprocess.run(
    ["docker", "push", f"{ecr_repo}:latest"],
    check=True
)

print(f"✅ Docker image successfully pushed to {ecr_repo}:latest")

Creating ECR repository: deep_research_scoping_agent...
ℹ️  ECR repository 'deep_research_scoping_agent' already exists
Authenticating to ECR...
Login Succeeded
Building Docker image: deep_research_scoping_agent...
Login Succeeded
Building Docker image: deep_research_scoping_agent...


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 884B done
#1 DONE 0.0s

#2 [internal] load metadata for ghcr.io/astral-sh/uv:python3.12-bookworm-slim
#2 DONE 0.6s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/7] FROM ghcr.io/astral-sh/uv:python3.12-bookworm-slim@sha256:b64bb25505ad7d6630f185a0f5ab29dab437a5b99f522885c1665842fb0c304b
#4 resolve ghcr.io/astral-sh/uv:python3.12-bookworm-slim@sha256:b64bb25505ad7d6630f185a0f5ab29dab437a5b99f522885c1665842fb0c304b 0.0s done
#4 DONE 0.0s

#5 [internal] load build context
#2 DONE 0.6s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/7] FROM ghcr.io/astral-sh/uv:python3.12-bookworm-slim@sha256:b64bb25505ad7d6630f185a0f5ab29dab437a5b99f522885c1665842fb0c304b
#4 resolve ghcr.io/astral-sh/uv:python3.12-bookworm-slim@sha256:b64bb25505ad7d6630f185a0f5ab29dab437a5b99f522885c1665842fb0

Tagging image for ECR: 482387069690.dkr.ecr.us-west-2.amazonaws.com/deep_research_scoping_agent:latest...
Pushing image to ECR: 482387069690.dkr.ecr.us-west-2.amazonaws.com/deep_research_scoping_agent:latest...
The push refers to repository [482387069690.dkr.ecr.us-west-2.amazonaws.com/deep_research_scoping_agent]
162e72af9357: Waiting
73c4402090ca: Waiting
2cc0a8e7ed73: Waiting
f53431374520: Waiting
05ccb1b61fbc: Waiting
f130974e425e: Waiting
be36e825b816: Waiting
a907c6775c52: Waiting
927305b24ea9: Waiting
469aebfb7b90: Waiting
e75f0adcf796: Waiting
801c156dc635: Waiting
927305b24ea9: Layer already exists
469aebfb7b90: Layer already exists
e75f0adcf796: Layer already exists
801c156dc635: Waiting
162e72af9357: Layer already exists
73c4402090ca: Layer already exists
2cc0a8e7ed73: Layer already exists
f53431374520: Layer already exists
05ccb1b61fbc: Layer already exists
f130974e425e: Layer already exists
be36e825b816: Layer already exists
a907c6775c52: Waiting
162e72af9357: Waiting
73c4

In [35]:
# Create CodeBuild execution role
import json

iam_client = boto_session.client("iam")
role_name = f"deep-research-codebuild-role-{region}"

# IAM trust policy for CodeBuild
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "codebuild.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# IAM inline policy for CodeBuild
codebuild_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "CloudWatchLogs",
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": f"arn:aws:logs:{region}:{account_id}:log-group:/aws/codebuild/*"
        },
        {
            "Sid": "ECRAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:BatchCheckLayerAvailability",
                "ecr:GetDownloadUrlForLayer",
                "ecr:BatchGetImage",
                "ecr:GetAuthorizationToken",
                "ecr:PutImage",
                "ecr:InitiateLayerUpload",
                "ecr:UploadLayerPart",
                "ecr:CompleteLayerUpload"
            ],
            "Resource": [
                f"arn:aws:ecr:{region}:{account_id}:repository/{agent_name}",
                "*"
            ]
        },
        {
            "Sid": "S3SourceAccess",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:GetObjectVersion"
            ],
            "Resource": "arn:aws:s3:::*/*"
        }
    ]
}

# Create role
try:
    print(f"Creating IAM role: {role_name}...")
    create_role_response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description="CodeBuild execution role for Bedrock AgentCore"
    )
    role_arn = create_role_response["Role"]["Arn"]
    print(f"✅ IAM role created: {role_arn}")
    
    # Attach inline policy
    print("Attaching inline policy...")
    iam_client.put_role_policy(
        RoleName=role_name,
        PolicyName="CodeBuildPolicy",
        PolicyDocument=json.dumps(codebuild_policy)
    )
    print("✅ Inline policy attached successfully")
    
except iam_client.exceptions.EntityAlreadyExistsException:
    print(f"ℹ️  IAM role '{role_name}' already exists")
    role_arn = f"arn:aws:iam::{account_id}:role/{role_name}"
    
    # Update inline policy if role exists
    try:
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName="CodeBuildPolicy",
            PolicyDocument=json.dumps(codebuild_policy)
        )
        print("✅ Inline policy updated successfully")
    except Exception as e:
        print(f"⚠️  Warning: Could not update policy: {e}")

print(f"\nCodeBuild Role ARN: {role_arn}")

Creating IAM role: deep-research-codebuild-role-us-west-2...
ℹ️  IAM role 'deep-research-codebuild-role-us-west-2' already exists
✅ Inline policy updated successfully

CodeBuild Role ARN: arn:aws:iam::482387069690:role/deep-research-codebuild-role-us-west-2
ℹ️  IAM role 'deep-research-codebuild-role-us-west-2' already exists
✅ Inline policy updated successfully

CodeBuild Role ARN: arn:aws:iam::482387069690:role/deep-research-codebuild-role-us-west-2


In [36]:
# Since we already built and pushed the Docker image to ECR,
# we still need to configure but we'll use deployment_type="container"
# and provide the ECR repository to skip the Dockerfile generation

agentcore_runtime = Runtime()

agent_name = "deep_research_scoping_agent"

# Get AWS account ID and construct ECR image URI
account_id = boto_session.client("sts").get_caller_identity()["Account"]
ecr_repository = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{agent_name}"

response = agentcore_runtime.configure(
    entrypoint="agent_core/scoping_agent_core.py",
    auto_create_execution_role=True,
    code_build_execution_role=role_arn,  # Use the role we created
    auto_create_ecr=False,  # Don't create new ECR - use existing
    deployment_type="container",  # Use container deployment
    region=region,
    agent_name=agent_name,
    requirements_file="requirements.txt",
    ecr_repository=ecr_repository  # Use the ECR image we built and pushed
)
response

Entrypoint parsed: file=/workspaces/agent_core_deep_research/src/agent_core/scoping_agent_core.py, bedrock_agentcore_name=scoping_agent_core
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: deep_research_scoping_agent
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: deep_research_scoping_agent
Memory disabled
Network mode: PUBLIC
Generated Dockerfile: Dockerfile
Memory disabled
Network mode: PUBLIC
Generated Dockerfile: Dockerfile
Generated .dockerignore: /workspaces/agent_core_deep_research/src/.dockerignore
Keeping 'deep_research_scoping_agent' as default agent
Generated .dockerignore: /workspaces/agent_core_deep_research/src/.dockerignore
Keeping 'deep_research_scoping_agent' as default agent
Bedrock AgentCore configured: /workspaces/agent_core_deep_research/src/.bedrock_agentcore.yaml
Bedrock AgentCore configured: /workspaces/agent_core_deep_research/src/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/workspaces/agent_core_deep_research/src/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/workspaces/agent_core_deep_research/src/Dockerfile'), dockerignore_path=PosixPath('/workspaces/agent_core_deep_research/src/.dockerignore'), runtime='Docker', runtime_type=None, region='us-west-2', account_id='482387069690', execution_role=None, ecr_repository='482387069690.dkr.ecr.us-west-2.amazonaws.com/deep_research_scoping_agent', auto_create_ecr=False, s3_path=None, auto_create_s3=False, memory_id=None, network_mode='PUBLIC', network_subnets=None, network_security_groups=None, network_vpc_id=None)

In [37]:
# Launch using the existing ECR image (skip the build process)
# Since we already built and pushed the image, we don't want to rebuild
launch_result = agentcore_runtime.launch()

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'deep_research_scoping_agent' to account 482387069690 (us-west-2)
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'deep_research_scoping_agent' to account 482387069690 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Using ECR repository from config: 482387069690.dkr.ecr.us-

In [38]:
import boto3
from pprint import pprint

client = boto3.client('bedrock-agentcore-control', region_name='us-west-2')
res = client.list_agent_runtimes()
#pprint(res['agentRuntimes'])
agent_id = [item['agentRuntimeId'] for item in res['agentRuntimes'] if item['agentRuntimeName']==agent_name][0]

res = client.get_agent_runtime(agentRuntimeId=agent_id)
execution_role_arn = res['roleArn']


In [39]:
role_name = execution_role_arn.split('/')[-1]
ssm_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "SSMParameterStoreRead",
            "Effect": "Allow",
            "Action": [
                "ssm:GetParameter",
                "ssm:GetParameters",
                "ssm:GetParametersByPath"
            ],
            "Resource": f"arn:aws:ssm:us-west-2:482387069690:parameter/deep_research_scoping_agent/*"
        },
        {
            "Sid": "KMSDecrypt",
            "Effect": "Allow",
            "Action": ["kms:Decrypt"],
            "Resource": "*"
        }
    ]
}

iam_client = boto_session.client("iam")
iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName="SSMParameterStoreAccess",
    PolicyDocument=json.dumps(ssm_policy)
)

print(f"Updated role '{role_name}' with SSM Parameter Store access")


Updated role 'AmazonBedrockAgentCoreSDKRuntime-us-west-2-a59a4561a2' with SSM Parameter Store access


In [40]:
payload = {
    "prompt": "I want to research the best coffee shops in San Francisco.",
    "thread_id": "test-thread-1"
}
result = agentcore_runtime.invoke(payload)




In [42]:
pprint(result)

{'ResponseMetadata': {'HTTPHeaders': {'baggage': 'Self=1-6913d168-403711c66f70c4132154ec71,session.id=b439b380-9960-475a-87a6-677a4764a6d8',
                                      'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Wed, 12 Nov 2025 00:14:34 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amzn-bedrock-agentcore-runtime-session-id': 'b439b380-9960-475a-87a6-677a4764a6d8',
                                      'x-amzn-requestid': 'ba9454c6-7e38-4ec2-97be-a37ec7289854',
                                      'x-amzn-trace-id': 'Root=1-6913d168-6be76cb427015e4d0dfdd262;Parent=dd68debd26fe999e;Sampled=1;Self=1-6913d168-403711c66f70c4132154ec71'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'ba9454c6-7e38-4ec2-97be-a37ec7289854',
                      'RetryAttempts': 0},
 'bagga